<a href="https://colab.research.google.com/github/DrigoDomingos/onfarm/blob/master/onfarm_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip Etapa1.zip

Archive:  Etapa1.zip
   creating: Etapa1/AzulClaro/
  inflating: Etapa1/AzulClaro/1.jpg  
  inflating: Etapa1/AzulClaro/1.xml  
  inflating: Etapa1/AzulClaro/2.jpg  
  inflating: Etapa1/AzulClaro/2.xml  
  inflating: Etapa1/AzulClaro/3.jpg  
  inflating: Etapa1/AzulClaro/3.xml  
  inflating: Etapa1/AzulClaro/4.jpg  
  inflating: Etapa1/AzulClaro/4.xml  
  inflating: Etapa1/AzulClaro/5.jpg  
  inflating: Etapa1/AzulClaro/5.xml  


# Convert XML to CSV

In [0]:
import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile

annotations = glob('/content/Etapa1/AzulClaro/*.xml')

df = []
cnt = 0
for file in annotations:
    #filename = file.split('/')[-1].split('.')[0] + '.jpg'
    #filename = str(cnt) + '.jpg'
    filename = file.split('\\')[-1]
    filename =filename.split('.')[0] + '.jpg'
    row = []
    parsedXML = ET.parse(file)
    for node in parsedXML.getroot().iter('object'):
        blood_cells = node.find('name').text
        xmin = int(node.find('bndbox/xmin').text)
        xmax = int(node.find('bndbox/xmax').text)
        ymin = int(node.find('bndbox/ymin').text)
        ymax = int(node.find('bndbox/ymax').text)

        row = [filename, blood_cells, xmin, xmax, ymin, ymax]
        df.append(row)
        cnt += 1

data = pd.DataFrame(df, columns=['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])

data[['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('test.csv', index=False)

In [6]:
!cat test.csv


filename,cell_type,xmin,xmax,ymin,ymax
/content/Etapa1/AzulClaro/2.jpg,azul_claro,683,737,538,584
/content/Etapa1/AzulClaro/2.jpg,azul_claro,1205,1251,498,544
/content/Etapa1/AzulClaro/2.jpg,azul_claro,1116,1175,841,890
/content/Etapa1/AzulClaro/2.jpg,azul_claro,735,783,744,792
/content/Etapa1/AzulClaro/2.jpg,azul_claro,1597,1648,854,892
/content/Etapa1/AzulClaro/2.jpg,azul_claro,1348,1397,568,614
/content/Etapa1/AzulClaro/2.jpg,azul_claro,851,900,841,903
/content/Etapa1/AzulClaro/2.jpg,azul_claro,618,672,638,698
/content/Etapa1/AzulClaro/2.jpg,azul_claro,600,656,722,765
/content/Etapa1/AzulClaro/2.jpg,azul_claro,772,824,441,506
/content/Etapa1/AzulClaro/2.jpg,azul_claro,1559,1629,679,749
/content/Etapa1/AzulClaro/2.jpg,azul_claro,1010,1072,682,736
/content/Etapa1/AzulClaro/1.jpg,azul_claro,1235,1270,1030,1065
/content/Etapa1/AzulClaro/1.jpg,azul_claro,1605,1643,930,984
/content/Etapa1/AzulClaro/1.jpg,azul_claro,1208,1262,303,354
/content/Etapa1/AzulClaro/1.jpg,azul_claro,1400,1462,98,